In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# MobileNet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Set device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define MobileNetV3 model
model = models.mobilenet_v3_large(pretrained=False, num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Train the model
num_epochs = 200
for epoch in range(num_epochs):
    # Train the model
    model.train()
    train_loss = 0
    train_correct = 0
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Test the model
    model.eval()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)
            test_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Print results for this epoch
    train_loss /= len(trainset)
    train_acc = train_correct / len(trainset)
    test_loss /= len(testset)
    test_acc = test_correct / len(testset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'mobilenetv3_cifar10.pth')


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/200], Train Loss: 1.7683, Train Acc: 0.3296, Test Loss: 2.1071, Test Acc: 0.2367
Epoch [2/200], Train Loss: 1.5482, Train Acc: 0.4267, Test Loss: 2.0478, Test Acc: 0.3681
Epoch [3/200], Train Loss: 1.4657, Train Acc: 0.4664, Test Loss: 1.5436, Test Acc: 0.4461
Epoch [4/200], Train Loss: 1.4351, Train Acc: 0.4767, Test Loss: 1.7409, Test Acc: 0.3911
Epoch [5/200], Train Loss: 1.4071, Train Acc: 0.4871, Test Loss: 1.3712, Test Acc: 0.5000
Epoch [6/200], Train Loss: 1.5535, Train Acc: 0.4284, Test Loss: 1.4708, Test Acc: 0.4581
Epoch [7/200], Train Loss: 1.7065, Train Acc: 0.3699, Test Loss: 3.2442, Test Acc: 0.1830
Epoch [8/200], Train Loss: 1.5843, Train Acc: 0.4141, Test Loss: 1.4705, Test Acc: 0.4496
Epoch [9/200], Train Loss: 1.4509, Train Acc: 0.4675, Test Loss: 1.4435, Test Acc: 0.4668
Epoch [10/200], Train Loss: 1.4947, Train Acc: 0.4513, Test Loss: 1.8074, Test Acc: 0.3662
Epoch [11/200], Train Loss: 1.4303, Train Acc: 0.4755, Test Loss: 1.3365, Test Acc: 0.5100
Epoch [1

## VGG-16

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Set device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define VGG16 model
class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Set device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define VGG16 model
model = models.vgg16(pretrained=False, num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Train the model
num_epochs = 200
for epoch in range(num_epochs):
    # Train the model
    model.train()
    train_loss = 0
    train_correct = 0
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Test the model
    model.eval()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)
            test_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Print results for this epoch
    train_loss /= len(trainset)
    train_acc = train_correct / len(trainset)
    test_loss /= len(testset)
    test_acc = test_correct / len(testset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'vgg16_cifar10.pth')


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/200], Train Loss: 2.0651, Train Acc: 0.1980, Test Loss: 1.8895, Test Acc: 0.2341
Epoch [2/200], Train Loss: 1.6476, Train Acc: 0.3668, Test Loss: 1.3698, Test Acc: 0.4872
Epoch [3/200], Train Loss: 1.3697, Train Acc: 0.4968, Test Loss: 1.1837, Test Acc: 0.5623
Epoch [4/200], Train Loss: 1.1515, Train Acc: 0.5835, Test Loss: 0.9715, Test Acc: 0.6455
Epoch [5/200], Train Loss: 1.0042, Train Acc: 0.6444, Test Loss: 0.9004, Test Acc: 0.6852
Epoch [6/200], Train Loss: 0.8863, Train Acc: 0.6874, Test Loss: 0.9415, Test Acc: 0.6665
Epoch [7/200], Train Loss: 0.8087, Train Acc: 0.7169, Test Loss: 0.7593, Test Acc: 0.7406
Epoch [8/200], Train Loss: 0.7298, Train Acc: 0.7464, Test Loss: 0.6944, Test Acc: 0.7621
Epoch [9/200], Train Loss: 0.6648, Train Acc: 0.7723, Test Loss: 0.6350, Test Acc: 0.7801
Epoch [10/200], Train Loss: 0.6169, Train Acc: 0.7905, Test Loss: 0.5915, Test Acc: 0.8048
Epoch [11/200], Train Loss: 0.5759, Train Acc: 0.8044, Test Loss: 0.6763, Test Acc: 0.7765
Epoch [1